# Project Goals

1. Want to use Node2Vec for drug-target & pathway sensitivity
2. Want to focus on genes, CCL 

### External References

G2Vec: Distributed gene representations for identification of cancer prognostic genes
* https://www.nature.com/articles/s41598-018-32180-0

* https://psb.stanford.edu/psb-online/proceedings/psb20/Pershad.pdf
* https://www.nature.com/articles/s41598-019-38793-3.pdf
* https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8401331&tag=1
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6311944/
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6454041/
* http://snap.stanford.edu/deepnetbio-ismb/ipynb/Human+Disease+Network.html
* https://towardsdatascience.com/node2vec-embeddings-for-graph-data-32a866340fef
* https://www.kaggle.com/ferdzso/knowledge-graph-analysis-with-node2vec
* https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007607


buliding of omics data needs to bias the random walk of the node2vec, by subsetting the input space per omics.
https://www.nature.com/articles/nmeth.2651
https://networkx.github.io/documentation/stable/tutorial.html#what-to-use-as-nodes-and-edges

https://datastorm-open.github.io/visNetwork/more.html

what is a node?

Databases:
https://amp.pharm.mssm.edu/X2K/#datasets
https://depmap.org/portal/download/
https://amp.pharm.mssm.edu/Harmonizome/download

In [1]:
import os, json
import xml.etree.ElementTree as ET
import networkx as nx
import pandas as pd
import numpy as np
from py2neo import Graph, Node
from pprint import pprint

baseId = 0
#graph = Graph("bolt://localhost:7687/neo4j/OmicSuperGraph", auth=("neo4j", "ngs4"))
#graph = neo4j.GraphDatabaseService("http://localhost:7474/db/data/")
graph = Graph(host="localhost", name="OmicSuperGraph", auth=("neo4j", "ngs4"))

with open('data/HGNC_protein-coding_gene.json') as f:
  hgnc = json.load(f)



KeyError: "Graph 'OmicSuperGraph' does not exist for service 'bolt://neo4j@localhost:7687'"

In [55]:
# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(hgnc['response'].keys())
gene_num = int(hgnc['response']['numFound'])
print(len(hgnc['response']['docs']))
hgncByGeneSymbol = {}
hgncGeneGroups = {}
#hgncLocusGroups = {} # 'locus_group': 'protein-coding gene','locus_type': 'gene with protein product',
for record in hgnc['response']['docs']:
    ### Prepare the Gene Groups for future connectivity
    if 'gene_group_id' in record:
        for i in range(len(record['gene_group_id'])):
            if not record['gene_group_id'][i] in hgncGeneGroups:
                hgncGeneGroups[record['gene_group_id'][i]] = record['gene_group'][i]
    ### Make lookup map for HGNC records
    if 'symbol' in record:
        hgncByGeneSymbol[record['symbol']] = record
    
    
print(len(hgncGeneGroups.keys()))

#pprint(record)


dict_keys(['numFound', 'docs', 'start'])
19301
1293


In [56]:
#from https://github.com/harveyl888/neo4jKEGG
def read_kegg_xml(filename):
    compounds = list()
    genes = list()
    maps = list()
    rels = list()
    tree = ET.parse(filename).getroot()
    pathway_number = tree.attrib['number']
    pathway_name = tree.attrib['title']
    reactions_xml = tree.findall("entry")
    for r in reactions_xml:
        reaction = dict()
        reaction['type'] = r.attrib['type']
        if reaction['type'] == 'group':
            continue
        reaction['id'] = r.attrib['id']
        reaction['name'] = [x for x in r.attrib['name'].split(" ")]
        reaction['url'] = r.attrib['link']
        tmpS = r.find('graphics').get('name')
        if tmpS is not None:
            reaction['symbol'] = [x for x in tmpS.replace('...','').split(" ")]
        
        if reaction['type'] == 'compound':
            compounds.append(reaction)
        elif reaction['type'] == 'gene':
            genes.append(reaction)
        elif reaction['type'] == 'map':
            maps.append(reaction)
        else:
            print("UNEXPECTED TYPE: "+reaction['type'])
    ### Getting relationships
    reactions_xml = tree.findall("relation")
    for r in reactions_xml:
        reaction = dict()
        reaction['node1'] = r.attrib['entry1']
        reaction['node2'] = r.attrib['entry2']
        reaction['level'] = r.attrib['type']
        reaction['evidence'] = r.find('subtype').get('name')
        reaction['direction'] = r.find('subtype').get('value')
        rels.append(reaction)

    return {'compounds':compounds,'genes':genes,'maps':maps,'relationships':rels}


def load_compounds(cList):
       # pprint(cRec)
       # print(type(cRec))
        graph.create(Node("Compound",kegg_id=cRec['id'],symbol=cRec['symbol'],kegg_names=cRec['name']))
    

p1 = read_kegg_xml('data/kegg/hsa05200.xml')
# pprint(p1['compounds'])
#np.unique([x['type'] for x in p1])


# load_compounds(p1['compounds'])


{'id': '221',
 'name': ['cpd:C00951',
          'cpd:C00410',
          'cpd:C07653',
          'cpd:C01227',
          'cpd:C00280',
          'cpd:C00735',
          'cpd:C00762'],
 'symbol': ['C00951'],
 'type': 'compound',
 'url': 'https://www.kegg.jp/dbget-bin/www_bget?C00951+C00410+C07653+C01227+C00280+C00735+C00762'}
<class 'dict'>
{'id': '222',
 'name': ['cpd:C00187'],
 'symbol': ['C00187'],
 'type': 'compound',
 'url': 'https://www.kegg.jp/dbget-bin/www_bget?C00187'}
<class 'dict'>
{'id': '223',
 'name': ['cpd:C05981'],
 'symbol': ['C05981'],
 'type': 'compound',
 'url': 'https://www.kegg.jp/dbget-bin/www_bget?C05981'}
<class 'dict'>
{'id': '224',
 'name': ['cpd:C00165'],
 'symbol': ['C00165'],
 'type': 'compound',
 'url': 'https://www.kegg.jp/dbget-bin/www_bget?C00165'}
<class 'dict'>
{'id': '225',
 'name': ['cpd:C01245'],
 'symbol': ['C01245'],
 'type': 'compound',
 'url': 'https://www.kegg.jp/dbget-bin/www_bget?C01245'}
<class 'dict'>
{'id': '226',
 'name': ['cpd:C00076'],


AttributeError: 'Graph' object has no attribute 'open_browser'